In [7]:
%%time
import os
import json

import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.utils import unique_name_from_base

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()
prefix = 'wenet220'
output_path = f"s3://{bucket}/{prefix}"

print("boto3.__version__:{}".format(boto3.__version__))
print("sagemaker.__version__:{}".format(sagemaker.__version__))
print("bucket:{}".format(bucket))
print("role:{}".format(role))

boto3.__version__:1.26.96
sagemaker.__version__:2.140.1
bucket:sagemaker-us-east-1-348052051973
role:arn:aws:iam::348052051973:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole
CPU times: user 394 ms, sys: 59 ms, total: 453 ms
Wall time: 871 ms


In [8]:
%%markdown
Change the /root/wenet to /opt/ml/input in all data.list files (especially for train_960 and dev)
The "Librispeech" in data.list file in Github has the wrong captalization because it's wrong when I upload it. Please change it yourself!

Change the /root/wenet to /opt/ml/input in all data.list files (especially for train_960 and dev)
The "Librispeech" in data.list file in Github has the wrong captalization because it's wrong when I upload it. Please change it yourself!


In [32]:
from sagemaker.inputs import TrainingInput
prefix_dataset = "wenet220/export"
loc =f"s3://{bucket}/{prefix_dataset}"

training = TrainingInput(
    s3_data_type='S3Prefix', # Available Options: S3Prefix | ManifestFile | AugmentedManifestFile
    s3_data=loc,
    distribution='FullyReplicated', # Available Options: FullyReplicated | ShardedByS3Key 
    input_mode='FastFile'
)

In [33]:
instance_type = "ml.p4d.24xlarge"
# instance_type='local'

max_run = 432000
checkpoint_s3_uri = f"s3://{bucket}/{prefix}/checkpoints"

hyperparameters = {
    'datadir':'/opt/ml/input/data/training',
    'stage': '4',
    'stop_stage': '4',
    'train_config': 'conf/train_conformer.yaml',
    'model_dir': '/opt/ml/model',
    'checkpoint_dir': '/opt/ml/checkpoints',
    'output_dir': '/opt/ml/output/data',
}

est = PyTorch(
    entry_point="run-librispeech.sh",
    source_dir="/root/wenet-source/wenet",
    framework_version="1.11.0",
    py_version="py38",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size=200,
    disable_profiler=True,
    debugger_hook_config=False,
    base_job_name=prefix,
    hyperparameters = hyperparameters,
    checkpoint_s3_uri = checkpoint_s3_uri,
    output_path = f"s3://{bucket}/{prefix}/",
    # keep_alive_period_in_seconds=1800,
    max_run = max_run,
    tags = [{"Key": "team", "Value": "asr"}, {"Key": "person", "Value": "andrew"}, {"Key": "project", "Value": "abc"}],
)

In [ ]:
%%time
job_name = est.fit({"training":training})
#job_name = est.fit()

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: wenet220-2023-03-29-03-16-01-482


2023-03-29 03:17:23 Starting - Starting the training job...
2023-03-29 03:18:00 Starting - Preparing the instances for training.........
2023-03-29 03:19:30 Downloading - Downloading input data......
2023-03-29 03:20:21 Training - Downloading the training image..................
2023-03-29 03:23:27 Training - Training image download completed. Training in progress......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-03-28 23:24:13,504 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-03-28 23:24:13,566 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-03-28 23:24:13,575 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-03-28 23:24:13,577 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-03-28 23:24:19,523 sagemaker-training-toolkit INFO     No Neurons det

In [7]:
model_data = est.model_data
print("Model artifact saved at:\n", model_data)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


Model artifact saved at:
 s3://sagemaker-us-east-1-348052051973/wenet/wenet-2022-12-08-05-36-25-104/output/model.tar.gz
